In [1]:
import geemap
import folium
import ee

# Initialize Earth Engine
ee.Initialize(project='agb-sarbanibhadra')

# Define the Area of Interest (AOI)
aoi = ee.Geometry.Polygon([
    [
        [-73.0, -15.0],
        [-53.0, -15.0],
        [-53.0, 5.0],
        [-73.0, 5.0]
    ]
])

# Load Sentinel-1 data
s1 = ee.ImageCollection('COPERNICUS/S1_GRD') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-01-31') \
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV')) \
    .select('VV') \
    .mean()

# Load Sentinel-2 data
s2 = ee.ImageCollection('COPERNICUS/S2_HARMONIZED') \
    .filterBounds(aoi) \
    .filterDate('2023-01-01', '2023-01-31') \
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20)) \
    .median()

# Compute NDVI from Sentinel-2
ndvi = s2.normalizedDifference(['B8', 'B4']).rename('NDVI')

# Load DEM data
dem = ee.Image('USGS/SRTMGL1_003').clip(aoi)

# Compute slope from DEM
slope = ee.Terrain.slope(dem).rename('Slope')

# Combine features into a single image
combined = s1.addBands(ndvi).addBands(slope)

# Dummy AGB model function (Replace with your model)
def predict_agb(image):
    return image.select('NDVI').multiply(100).rename('AGB')

# Apply the AGB model
agb = predict_agb(combined)

# Load REDD+ projects data from Earth Engine assets
# redd_projects = ee.FeatureCollection('users/yourusername/yourassetname')

# Create a Map object
map = geemap.Map(center=[-5.0, -63.0], zoom=5)

# Add the AGB layer to the map
map.addLayer(agb, {
    'min': 0,
    'max': 200,
    'palette': ['blue', 'green', 'yellow', 'red']
}, 'AGB')

# Add the AOI boundary
map.addLayer(aoi, {}, 'AOI Boundary')

# Add REDD+ projects layer to the map
# map.addLayer(redd_projects, {
#     'color': 'red',
#     'opacity': 0.5
# }, 'REDD+ Projects')

# Add layer controls
map.addLayerControl()

# Display the map
map

# Save the map as an HTML file
map.save('agb_map_with_redd_projects.html')



Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.
Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [2]:
map

Map(center=[-5.0, -63.0], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGUI…